In [ ]:
!pip install vosk
!pip install pydub
!pip install autokeras

In [ ]:
import os
import pandas as pd
from pydub import AudioSegment
from google.colab import files, drive

import autokeras as ak
from sklearn.model_selection import train_test_split

from vosk import Model, KaldiRecognizer, SetLogLevel
import subprocess
import json

Using TensorFlow backend


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Для vosk-модели

SetLogLevel(0)

# Проверяем наличие модели
if not os.path.exists("/content/drive/My Drive/Media_108/Vosk_модель/vosk-model-ru-0.42"):
    print ("Please download the model from https://alphacephei.com/vosk/models and unpack as 'model' in the current folder.")
    exit (1)

# Устанавливаем Frame Rate
FRAME_RATE = 8000
CHANNELS=1

vosk_model = Model("/content/drive/My Drive/Media_108/Vosk_модель/vosk-model-ru-0.42")
rec = KaldiRecognizer(vosk_model, FRAME_RATE)
rec.SetWords(True)

In [ ]:
# Функция для преобразования аудио в текст
def audio_to_text(audio_file):
    SetLogLevel(0)

    # Используя библиотеку pydub делаем предобработку аудио
    mp3 = AudioSegment.from_mp3(audio_file)
    mp3 = mp3.set_channels(CHANNELS)
    mp3 = mp3.set_frame_rate(FRAME_RATE)

    # Преобразуем вывод в json
    rec.AcceptWaveform(mp3.raw_data)
    result = rec.Result()
    text = json.loads(result)["text"]

    return text

In [ ]:
# Преобразование аудио в текст и запись в файлы .csv
for folder in os.listdir("/content/drive/MyDrive/Media_108/CallS"):
    print(folder)
    for file in os.listdir(os.path.join("/content/drive/MyDrive/Media_108/CallS", folder)):
        print(file)
        audio_text = audio_to_text(os.path.join("/content/drive/MyDrive/Media_108/CallS", folder, file))

        # Добавление преобразованного текста в датафрейм
        df = pd.read_csv(os.path.join("/content/drive/MyDrive/Media_108/Информация о звонках", "Tags", f"Headliner_{folder}_tags_to_identifiers.csv"))
        df['Текст'] = audio_text
        df.to_csv(os.path.join("/content/drive/MyDrive/Media_108/Информация о звонках", "Tags", f"Headliner_{folder}_tags_to_identifiers.csv"), index=False)

target_filtered
3083491117.mp3
3085130850.mp3
3060317698.mp3
3047451656.mp3
3078147295.mp3
3096154027.mp3


In [ ]:
# Создание обучающей и тестовой выборки
data = pd.concat([pd.read_csv("Headliner_target_filtered_tags_to_identifiers.csv"),
                  pd.read_csv("Headliner_second_target_filtered_tags_to_identifiers.csv"),
                  pd.read_csv("Headliner_non_target_filtered_tags_to_identifiers.csv")])


In [ ]:
# Сохранение датафрейма с преобразованным текстом
data.to_csv("transformed_audio_text.csv", index=False)

In [ ]:
X = data['Теги']
y = data['Текст']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Используем Auto-Keras для обучения нейронной сети
model = ak.TextClassifier(max_trials=10)  # Пример параметра max_trials
model.fit(X_train, y_train)

In [ ]:
# Сохранение обученной модели
model.export_model().save("trained_model", save_format="tf")